In [1]:
import pandas as pd
import numpy as np
import regex as re
import swifter
import operator

In [2]:
#comments = pd.read_csv("comments_with_topics.csv")
posts = pd.read_csv("posts_with_topics.csv")
comments = pd.read_csv("comments_with_topics.csv")

In [5]:
posts = posts.iloc[:,1:]
display(posts)

,0,1,2,3,4
0,"(0, 0.011203291)","(1, 0.0111992275)","(2, 0.6498991)","(3, 0.3161924)","(4, 0.011505915)"
1,"(0, 0.17918065)","(2, 0.6070125)","(3, 0.2096066)",NaN,NaN
2,"(2, 0.30315173)","(3, 0.678872)",NaN,NaN,NaN
3,"(0, 0.013800318)","(1, 0.013549726)","(2, 0.5832353)","(3, 0.37565556)","(4, 0.01375903)"
4,"(0, 0.015556041)","(1, 0.01550649)","(2, 0.015572741)","(3, 0.93762773)","(4, 0.01573696)"
...,...,...,...,...,...
53422,"(0, 0.012782481)","(1, 0.013101777)","(2, 0.80132765)","(3, 0.15984458)","(4, 0.012943519)"
53423,"(0, 0.1289561)","(2, 0.85926026)",NaN,NaN,NaN
53424,"(0, 0.05156757)","(1, 0.3155836)","(2, 0.05158823)","(3, 0.5298284)","(4, 0.051432196)"
53425,"(0, 0.03369619)","(1, 0.03402428)","(2, 0.033827446)","(3, 0.8640226)","(4, 0.034429472)"


In [15]:
comment_ids = comments['id']
comments = comments.iloc[:,1:-1]
display(comments)
#display(posts)

,0,1,2,3,4,5,6,7,8,9
0,"(0, 0.41379768)","(1, 0.3051304)","(2, 0.012508093)","(3, 0.012504478)","(4, 0.012503766)","(5, 0.012504545)","(6, 0.19353274)","(7, 0.012507756)","(8, 0.0125039)","(9, 0.012506664)"
1,"(0, 0.77496153)","(1, 0.025004435)","(2, 0.0250039)","(3, 0.025003418)","(4, 0.025003418)","(5, 0.025003418)","(6, 0.025009612)","(7, 0.025003418)","(8, 0.025003418)","(9, 0.025003418)"
2,"(0, 0.020016339)","(1, 0.3402465)","(2, 0.25150862)","(3, 0.020019569)","(4, 0.020014852)","(5, 0.26812732)","(6, 0.020014834)","(7, 0.020016912)","(8, 0.020014834)","(9, 0.02002024)"
3,"(0, 0.03334265)","(1, 0.03333781)","(2, 0.033340894)","(3, 0.033335816)","(4, 0.033335816)","(5, 0.03336339)","(6, 0.033335816)","(7, 0.6999305)","(8, 0.033341467)","(9, 0.033335816)"
4,"(0, 0.03334009)","(1, 0.3665453)","(2, 0.03334009)","(3, 0.03334009)","(4, 0.36672753)","(5, 0.03334009)","(6, 0.033344593)","(7, 0.03334009)","(8, 0.03334009)","(9, 0.033342037)"
...,...,...,...,...,...,...,...,...,...,...
5217254,"(0, 0.016702672)","(1, 0.28887543)","(2, 0.18182828)","(3, 0.016701965)","(4, 0.016701965)","(5, 0.0167025)","(6, 0.016706241)","(7, 0.41237503)","(8, 0.016703917)","(9, 0.016701965)"
5217255,"(0, 0.36069873)","(1, 0.012536783)","(2, 0.012544638)","(3, 0.012539756)","(4, 0.012535005)","(5, 0.2611179)","(6, 0.012535318)","(7, 0.29042014)","(8, 0.012535005)","(9, 0.012536719)"
5217256,"(0, 0.012511537)","(1, 0.012511557)","(2, 0.012518098)","(3, 0.88738513)","(4, 0.012511789)","(5, 0.012511537)","(6, 0.012514349)","(7, 0.012511537)","(8, 0.012511537)","(9, 0.012512957)"
5217257,"(0, 0.19430603)","(2, 0.30789554)","(5, 0.3653103)","(8, 0.09910222)",NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
##clearly this format is not helpful. we need them sorted first and foremost - primary topic, secondary topic, and spread
temp = posts.copy(deep = True)
def find_values(row):
    temp = [0,0,0,0,0]
    for topic_tuple in row:
        #print(row)
        #print(topic_tuple)
        #print(type(topic_tuple))
        if pd.isna(topic_tuple):
            continue
        vals = tuple(str(topic_tuple).replace("(","").replace(")","").split(", "))
        temp[int(vals[0])] = vals[1]
    return temp

fixed_values = (find_values,temp)
fixed = temp.apply(lambda x: find_values(x), axis = 1)
list_of_lists = [row for row in fixed]
fixed_df = pd.DataFrame(data =list_of_lists, columns = [0,1,2,3,4])
display(fixed_df)
posts = fixed_df

,0,1,2,3,4
0,0.011203291,0.0111992275,0.6498991,0.3161924,0.011505915
1,0.17918065,0,0.6070125,0.2096066,0
2,0,0,0.30315173,0.678872,0
3,0.013800318,0.013549726,0.5832353,0.37565556,0.01375903
4,0.015556041,0.01550649,0.015572741,0.93762773,0.01573696
...,...,...,...,...,...
53422,0.012782481,0.013101777,0.80132765,0.15984458,0.012943519
53423,0.1289561,0,0.85926026,0,0
53424,0.05156757,0.3155836,0.05158823,0.5298284,0.051432196
53425,0.03369619,0.03402428,0.033827446,0.8640226,0.034429472


In [17]:
temp = comments.copy(deep = True)
def find_values_v2(row):
    temp = [0,0,0,0,0,0,0,0,0,0]
    for topic_tuple in row:
        #print(row)
        #print(topic_tuple)
        #print(type(topic_tuple))
        if pd.isna(topic_tuple):
            continue
        vals = tuple(str(topic_tuple).replace("(","").replace(")","").split(", "))
        temp[int(vals[0])] = vals[1]
    return temp

#fixed_values = (find_values,temp)
fixed = temp.apply(lambda x: find_values_v2(x), axis = 1)
list_of_lists = [row for row in fixed]
fixed_df = pd.DataFrame(data =list_of_lists, columns = [0,1,2,3,4,5,6,7,8,9])
display(fixed_df)
comments = fixed_df

,0,1,2,3,4,5,6,7,8,9
0,0.41379768,0.3051304,0.012508093,0.012504478,0.012503766,0.012504545,0.19353274,0.012507756,0.0125039,0.012506664
1,0.77496153,0.025004435,0.0250039,0.025003418,0.025003418,0.025003418,0.025009612,0.025003418,0.025003418,0.025003418
2,0.020016339,0.3402465,0.25150862,0.020019569,0.020014852,0.26812732,0.020014834,0.020016912,0.020014834,0.02002024
3,0.03334265,0.03333781,0.033340894,0.033335816,0.033335816,0.03336339,0.033335816,0.6999305,0.033341467,0.033335816
4,0.03334009,0.3665453,0.03334009,0.03334009,0.36672753,0.03334009,0.033344593,0.03334009,0.03334009,0.033342037
...,...,...,...,...,...,...,...,...,...,...
5217254,0.016702672,0.28887543,0.18182828,0.016701965,0.016701965,0.0167025,0.016706241,0.41237503,0.016703917,0.016701965
5217255,0.36069873,0.012536783,0.012544638,0.012539756,0.012535005,0.2611179,0.012535318,0.29042014,0.012535005,0.012536719
5217256,0.012511537,0.012511557,0.012518098,0.88738513,0.012511789,0.012511537,0.012514349,0.012511537,0.012511537,0.012512957
5217257,0.19430603,0,0.30789554,0,0,0.3653103,0,0,0.09910222,0


True


In [41]:
ids = [comment_id.replace("('","").replace("')","").replace("'","").split(", ") for comment_id in comment_ids]
display(ids)

[['gftdfnp', 't3_kcvkwx'],
 ['gftdfk8', 't3_kcvkwx'],
 ['gftdfh0', 't1_gftd073'],
 ['gftdfek', 't3_kcvkwx'],
 ['gftdfb7', 't3_kcvkwx'],
 ['gftdf7b', 't3_kcvkwx'],
 ['gftdf4z', 't1_gftcijz'],
 ['gftdf1s', 't3_kcvkwx'],
 ['gftdf0t', 't3_kcvkwx'],
 ['gftdf0e', 't3_kcvkwx'],
 ['gftdeu9', 't3_kcvkwx'],
 ['gftdest', 't1_gffzkfq'],
 ['gftdepw', 't1_gftc4c3'],
 ['gftdepu', 't1_gftdbdo'],
 ['gftdepk', 't3_kcvkwx'],
 ['gftdepj', 't3_kcvkwx'],
 ['gftden7', 't3_kcyv9o'],
 ['gftdemz', 't1_gftcrsw'],
 ['gftdekc', 't3_kcvkwx'],
 ['gftdeh6', 't3_kcvkwx'],
 ['gftdecn', 't3_kcioej'],
 ['gftdecb', 't3_kcvkwx'],
 ['gftdeb4', 't3_kcvkwx'],
 ['gftde8n', 't3_kcvkwx'],
 ['gftde7n', 't1_gftdcav'],
 ['gftde6t', 't1_gftd69w'],
 ['gftde6b', 't3_kcpp7o'],
 ['gftde38', 't3_kcvkwx'],
 ['gftde2n', 't1_gftcvko'],
 ['gftde2a', 't1_gftcydd'],
 ['gftddve', 't3_kcvkwx'],
 ['gftddpx', 't3_kcvkwx'],
 ['gftddpt', 't3_kcvkwx'],
 ['gftddom', 't3_kcvkwx'],
 ['gftddnz', 't1_gftd8pu'],
 ['gftddn7', 't3_kcvkwx'],
 ['gftddmk', 't3_

In [9]:
comments['post_id'] = [post_id[1] for post_id in ids]
comments['comment_id'] = [comment_id[0] for comment_id in ids]
posts_temp = pd.read_csv("posts_rm_short_removed.csv")
posts['id'] = posts_temp['id']

In [10]:
display(posts)

,0,1,2,3,4,id
0,0.011203291,0.0111992275,0.6498991,0.3161924,0.011505915,kcz0hf
1,0.17918065,0,0.6070125,0.2096066,0,kcyyo1
2,0,0,0.30315173,0.678872,0,kcyxbk
3,0.013800318,0.013549726,0.5832353,0.37565556,0.01375903,kcywij
4,0.015556041,0.01550649,0.015572741,0.93762773,0.01573696,kcyk2w
...,...,...,...,...,...,...
53422,0.012782481,0.013101777,0.80132765,0.15984458,0.012943519,kl8wxo
53423,0.1289561,0,0.85926026,0,0,kl8wbp
53424,0.05156757,0.3155836,0.05158823,0.5298284,0.051432196,kl8m4y
53425,0.03369619,0.03402428,0.033827446,0.8640226,0.034429472,kl8f6l


In [47]:
display(comments)

,0,1,2,3,4,5,6,7,8,9,post_id,comment_id
0,0.41379768,0.3051304,0.012508093,0.012504478,0.012503766,0.012504545,0.19353274,0.012507756,0.0125039,0.012506664,t3_kcvkwx,gftdfnp
1,0.77496153,0.025004435,0.0250039,0.025003418,0.025003418,0.025003418,0.025009612,0.025003418,0.025003418,0.025003418,t3_kcvkwx,gftdfk8
2,0.020016339,0.3402465,0.25150862,0.020019569,0.020014852,0.26812732,0.020014834,0.020016912,0.020014834,0.02002024,t1_gftd073,gftdfh0
3,0.03334265,0.03333781,0.033340894,0.033335816,0.033335816,0.03336339,0.033335816,0.6999305,0.033341467,0.033335816,t3_kcvkwx,gftdfek
4,0.03334009,0.3665453,0.03334009,0.03334009,0.36672753,0.03334009,0.033344593,0.03334009,0.03334009,0.033342037,t3_kcvkwx,gftdfb7
...,...,...,...,...,...,...,...,...,...,...,...,...
5217254,0.016702672,0.28887543,0.18182828,0.016701965,0.016701965,0.0167025,0.016706241,0.41237503,0.016703917,0.016701965,t1_gkdmaa1,gkdmwie
5217255,0.36069873,0.012536783,0.012544638,0.012539756,0.012535005,0.2611179,0.012535318,0.29042014,0.012535005,0.012536719,t1_gkceomw,gkdmw91
5217256,0.012511537,0.012511557,0.012518098,0.88738513,0.012511789,0.012511537,0.012514349,0.012511537,0.012511537,0.012512957,t3_l3c0hj,gkdmw6k
5217257,0.19430603,0,0.30789554,0,0,0.3653103,0,0,0.09910222,0,t1_gkbyl29,gkdmvxa


In [11]:
posts.to_csv("posts_with_ids_topics.csv")
comments.to_csv("comments_with_ids_topics.csv")

DONE HERE ~~~~ NO NEED TO CONTINUE EXCEPT EXTRA CODE FOR OTHER USES

In [119]:
print(len(top_topics_from_comments[:][0]))
print(top_topics_from_comments[:][0][1])
top_topic =[]
top_topic_prop = []
secondary_topic = []
secondary_topic_prop = []
def extract_topics(tuple_list):
    #print(tuple_list)
    #print(type(tuple_list))
    top_topic.append(tuple_list[0][0])
    top_topic_prop.append(tuple_list[0][1])
    try:
        secondary_topic.append(tuple_list[1][0])
        secondary_topic_prop.append(tuple_list[1][1])
    except IndexError:
        secondary_topic.append(tuple_list[0][0])
        secondary_topic_prop.append(tuple_list[0][1])
    return 1

temp = [extract_topics(comment) for comment in top_topics_from_comments]
display(top_topic)
display(top_topic_prop)

10
('1', '0.3051304')


['0',
 '0',
 '1',
 '7',
 '4',
 '8',
 '7',
 '2',
 '0',
 '7',
 '3',
 '7',
 '3',
 '6',
 '0',
 '7',
 '7',
 '0',
 '0',
 '7',
 '2',
 '0',
 '6',
 '7',
 '5',
 '2',
 '4',
 '0',
 '0',
 '3',
 '3',
 '0',
 '5',
 '1',
 '3',
 '6',
 '0',
 '2',
 '8',
 '3',
 '5',
 '9',
 '1',
 '8',
 '4',
 '0',
 '2',
 '7',
 '7',
 '8',
 '0',
 '0',
 '0',
 '0',
 '7',
 '6',
 '3',
 '3',
 '5',
 '7',
 '6',
 '8',
 '5',
 '7',
 '3',
 '6',
 '5',
 '7',
 '4',
 '8',
 '9',
 '2',
 '9',
 '9',
 '9',
 '2',
 '3',
 '8',
 '2',
 '5',
 '7',
 '0',
 '9',
 '1',
 '3',
 '5',
 '2',
 '0',
 '1',
 '9',
 '9',
 '5',
 '3',
 '2',
 '0',
 '7',
 '9',
 '0',
 '6',
 '0',
 '2',
 '5',
 '2',
 '2',
 '1',
 '0',
 '6',
 '5',
 '5',
 '0',
 '3',
 '7',
 '9',
 '0',
 '3',
 '6',
 '3',
 '2',
 '5',
 '3',
 '3',
 '3',
 '0',
 '7',
 '6',
 '3',
 '2',
 '2',
 '9',
 '6',
 '9',
 '2',
 '3',
 '0',
 '2',
 '3',
 '1',
 '3',
 '1',
 '9',
 '9',
 '5',
 '5',
 '7',
 '6',
 '0',
 '4',
 '2',
 '0',
 '6',
 '3',
 '4',
 '9',
 '7',
 '0',
 '3',
 '9',
 '5',
 '7',
 '0',
 '1',
 '3',
 '8',
 '2',
 '9',
 '6',
 '3'

['0.41379768',
 '0.77496153',
 '0.3402465',
 '0.6999305',
 '0.36672753',
 '0.35026112',
 '0.31761363',
 '0.28117692',
 '0.6087006',
 '0.5251342',
 '0.28378785',
 '0.7748771',
 '0.552381',
 '0.5737195',
 '0.793154',
 '0.51041764',
 '0.7748955',
 '0.1',
 '0.5332141',
 '0.5574471',
 '0.53913677',
 '0.41999927',
 '0.5832801',
 '0.7749304',
 '0.43938887',
 '0.52474713',
 '0.20383842',
 '0.77474296',
 '0.7748989',
 '0.31722203',
 '0.6066257',
 '0.32102507',
 '0.3357191',
 '0.42002216',
 '0.3228661',
 '0.89996564',
 '0.36666858',
 '0.42005917',
 '0.36680678',
 '0.5265865',
 '0.7748533',
 '0.65249777',
 '0.46285212',
 '0.9857135',
 '0.36683112',
 '0.59582907',
 '0.45353615',
 '0.5252659',
 '0.5497534',
 '0.9857135',
 '0.1',
 '0.774925',
 '0.39716527',
 '0.6999548',
 '0.40864214',
 '0.51644504',
 '0.34758312',
 '0.5527148',
 '0.7749123',
 '0.35426828',
 '0.65151453',
 '0.9857135',
 '0.5733086',
 '0.81996757',
 '0.6990344',
 '0.27633727',
 '0.8499517',
 '0.6082792',
 '0.3784809',
 '0.9857135',
 

In [18]:
print(len(top_topics_from_posts[:][0]))
print(top_topics_from_posts[:][0][1])
top_topic =[]
top_topic_prop = []
secondary_topic = []
secondary_topic_prop = []
def extract_topics(tuple_list):
    #print(tuple_list)
    #print(type(tuple_list))
    top_topic.append(tuple_list[0][0])
    top_topic_prop.append(tuple_list[0][1])
    try:
        secondary_topic.append(tuple_list[1][0])
        secondary_topic_prop.append(tuple_list[1][1])
    except IndexError:
        secondary_topic.append(tuple_list[0][0])
        secondary_topic_prop.append(tuple_list[0][1])
    return 1

temp = [extract_topics(comment) for comment in top_topics_from_posts]
display(top_topic)
display(top_topic_prop)

5
('3', '0.3161924')


['2',
 '2',
 '3',
 '2',
 '3',
 '2',
 '3',
 '3',
 '2',
 '3',
 '3',
 '3',
 '3',
 '2',
 '0',
 '3',
 '4',
 '2',
 '1',
 '3',
 '0',
 '3',
 '3',
 '3',
 '3',
 '2',
 '2',
 '3',
 '3',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '3',
 '2',
 '2',
 '3',
 '2',
 '3',
 '2',
 '2',
 '2',
 '3',
 '3',
 '4',
 '3',
 '2',
 '2',
 '2',
 '3',
 '3',
 '2',
 '2',
 '2',
 '2',
 '4',
 '2',
 '3',
 '2',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '2',
 '2',
 '3',
 '3',
 '2',
 '2',
 '2',
 '3',
 '3',
 '3',
 '3',
 '3',
 '2',
 '3',
 '2',
 '2',
 '3',
 '2',
 '3',
 '2',
 '3',
 '3',
 '3',
 '2',
 '2',
 '2',
 '0',
 '3',
 '0',
 '3',
 '2',
 '3',
 '3',
 '3',
 '2',
 '3',
 '2',
 '1',
 '1',
 '2',
 '2',
 '3',
 '2',
 '2',
 '2',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '1',
 '3',
 '3',
 '1',
 '2',
 '1',
 '0',
 '2',
 '2',
 '0',
 '2',
 '2',
 '2',
 '2',
 '2',
 '4',
 '1',
 '4',
 '3',
 '3',
 '3',
 '2',
 '3',
 '3',
 '2',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '2',
 '3',
 '3',
 '3',
 '4',
 '2',
 '4',
 '2',
 '2',
 '3',
 '2',
 '2',
 '2'

['0.6498991',
 '0.6070125',
 '0.678872',
 '0.5832353',
 '0.93762773',
 '0.70007235',
 '0.9759946',
 '0.96972',
 '0.42935568',
 '0.6051965',
 '0.6024045',
 '0.508896',
 '0.71382105',
 '0.9605631',
 '0.4366257',
 '0.9352812',
 '0.3654697',
 '0.9550638',
 '0.54487073',
 '0.9263187',
 '0.59081376',
 '0.9367209',
 '0.7308251',
 '0.57591754',
 '0.9709486',
 '0.88406587',
 '0.65788597',
 '0.6590319',
 '0.896976',
 '0.89816403',
 '0.6297722',
 '0.8054992',
 '0.9686792',
 '0.86489385',
 '0.90947884',
 '0.94610345',
 '0.92609465',
 '0.59548354',
 '0.72885686',
 '0.8974306',
 '0.5202103',
 '0.8788886',
 '0.599525',
 '0.9319374',
 '0.8649006',
 '0.9246668',
 '0.79554534',
 '0.60218287',
 '0.9800465',
 '0.9544368',
 '0.5078811',
 '0.8583428',
 '0.56285405',
 '0.9185497',
 '0.4410746',
 '0.55660796',
 '0.5686197',
 '0.5457554',
 '0.9485919',
 '0.6926793',
 '0.6921737',
 '0.92655003',
 '0.5428877',
 '0.7206726',
 '0.38666984',
 '0.66322',
 '0.8601545',
 '0.7567247',
 '0.630412',
 '0.48636213',
 '0.60

In [19]:
##check the work
print(len(top_topic))
print(len(top_topic_prop))
print(len(secondary_topic))
print(len(secondary_topic_prop))

53427
53427
53427
53427


In [20]:
#d = {"primary_topic" : top_topic, "primary_prop" : top_topic_prop, "secondary_topic" :secondary_topic,"secondary_prop":secondary_topic_prop}
#primary_secondary_comment_topics = pd.DataFrame(data = d)
#display(primary_secondary_comment_topics)

d = {"primary_topic" : top_topic, "primary_prop" : top_topic_prop, "secondary_topic" :secondary_topic,"secondary_prop":secondary_topic_prop}
primary_secondary_post_topics = pd.DataFrame(data = d)
display(primary_secondary_post_topics)

,primary_topic,primary_prop,secondary_topic,secondary_prop
0,2,0.6498991,3,0.3161924
1,2,0.6070125,3,0.2096066
2,3,0.678872,2,0.30315173
3,2,0.5832353,3,0.37565556
4,3,0.93762773,4,0.01573696
...,...,...,...,...
53422,2,0.80132765,3,0.15984458
53423,2,0.85926026,0,0.1289561
53424,3,0.5298284,1,0.3155836
53425,3,0.8640226,4,0.034429472


In [30]:
##dont loose this
posts = pd.read_csv("posts_rm_short_removed.csv")
primary_secondary_post_topics['id'] = posts['id']
#primary_secondary_comment_topics.to_csv("primary_secondary_comment_topics.csv")
primary_secondary_post_topics.to_csv("primary_secondary_post_topics.csv")

In [31]:
display(primary_secondary_post_topics)
#ids = comments['id'].apply(lambda x: str(x).replace("(","").replace(")","").split(", "))

,primary_topic,primary_prop,secondary_topic,secondary_prop,id
0,2,0.6498991,3,0.3161924,kcz0hf
1,2,0.6070125,3,0.2096066,kcyyo1
2,3,0.678872,2,0.30315173,kcyxbk
3,2,0.5832353,3,0.37565556,kcywij
4,3,0.93762773,4,0.01573696,kcyk2w
...,...,...,...,...,...
53422,2,0.80132765,3,0.15984458,kl8wxo
53423,2,0.85926026,0,0.1289561,kl8wbp
53424,3,0.5298284,1,0.3155836,kl8m4y
53425,3,0.8640226,4,0.034429472,kl8f6l


In [ ]:
primary_secondary_comment_topics['post_id'] = [post_id[1] for post_id in ids]
primary_secondary_comment_topics['comment_id'] = [comment_id[0] for comment_id in ids]

In [134]:
display(primary_secondary_comment_topics)
primary_secondary_comment_topics.to_csv("primary_secondary_comment_topics.csv")

,primary_topic,primary_prop,secondary_topic,secondary_prop,post_id,comment_id
0,0,0.41379768,1,0.3051304,'t3_kcvkwx','gftdfnp'
1,0,0.77496153,6,0.025009612,'t3_kcvkwx','gftdfk8'
2,1,0.3402465,5,0.26812732,'t1_gftd073','gftdfh0'
3,7,0.6999305,5,0.03336339,'t3_kcvkwx','gftdfek'
4,4,0.36672753,1,0.3665453,'t3_kcvkwx','gftdfb7'
...,...,...,...,...,...,...
5217254,7,0.41237503,1,0.28887543,'t1_gkdmaa1','gkdmwie'
5217255,0,0.36069873,7,0.29042014,'t1_gkceomw','gkdmw91'
5217256,3,0.88738513,2,0.012518098,'t3_l3c0hj','gkdmw6k'
5217257,5,0.3653103,2,0.30789554,'t1_gkbyl29','gkdmvxa'


Things arent perfect with this solution - ideally we want a value for each topic for each post. This will be a very inefficient process,
so we will just start it up and wait on it